In [ ]:
import requests
import json
import re

In [ ]:
api_url = "http://localhost:11434/api/generate"
model_data = {"model": "deepseek-r1:8b"}

In [ ]:
query_sql = (
    "WITH OM AS ( "
    "    SELECT * from Orders_Main" 
    ") "
    "SELECT "
    " o.order_id, "
    ",c.customer_id"
    ",c.first_name"
    ",c.last_name"
    ",od.product_id"
    ",p.product_name"
    ",p.price"
    ",od.quantity"
    ",s.shipment_id"
    ",s.shipment_date"
    ",a.address_line1"
    ",a.city"
    ",a.state"
    ",a.zip_code"
    ",pay.payment_id"
    ",pay.payment_date"
    ",pay.amount"
    "FROM "
    "OM o "
    "JOIN Customers c ON o.customer_id = c.customer_id "
    "JOIN Order_Details od ON o.order_id = od.order_id "
    "JOIN Products p ON od.product_id = p.product_id "
    "JOIN Shipments s ON o.order_id = s.order_id "
    "JOIN Addresses a ON c.address_id = a.address_id "
    "JOIN Payments pay ON o.order_id = pay.order_id "
    "WHERE "
    "c.customer_id IN ( "
    "        SELECT customer_id "
    "        FROM Customers_Main "
    "        WHERE state = 'California' ) AND"
    "p.price > ( "
    "        SELECT AVG(price) "
    "        FROM Products_Main ) AND"
    "o.order_date >= DATE('2024-01-01') "
    "ORDER BY "
    " o.order_id"
    ",od.product_id; "
)
prompt = (
    "Extrae solo los nombres de las tablas de esta consulta SQL y devuélvelos separados por comas, sin explicaciones ni otro texto: "
    "'"
    f"{query_sql}"
    "'."
    "Ten en cuenta las tablas que estén en subconsultas y CTE. "
    "Verifica que los nombres de las tablas no correspondan a alias."
)

payload = {**model_data, "prompt": prompt}
response = requests.post(api_url, json=payload)

In [ ]:
if response.status_code == 200:
    text_response=""
    i=0
    lines = response.text.splitlines()
    for line in lines:
        json_data = json.loads(line)
        text_response += json_data["response"]
        
    print("Respuesta completa:", text_response)

    print("Respuesta corta:", re.sub(r"<think>.*?</think>", "", text_response, flags=re.DOTALL)[1:])
else:
    print(f"Error en la solicitud: {response.status_code}, {response.text}")